In [78]:
#Checking if GPU is running or not

!nvidia-smi

Thu Mar 21 09:17:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              26W /  70W |   8325MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [79]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [80]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [81]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ml"

## Helsinki-NLP/opus-mt-en-hi model

source: https://huggingface.co/Helsinki-NLP/opus-mt-en-hi



# The Dataset

Source: https://huggingface.co/datasets/cfilt/iitb-english-hindi

In [82]:
import json
from datasets import DatasetDict, Dataset

# Assuming 'enmldata.json' is the name of your custom dataset file
custom_dataset_path = "/content/enmldata.json"  # Update the path accordingly

# Load your custom dataset from the local file
with open(custom_dataset_path, "r", encoding="utf-8") as file:
    custom_dataset = json.load(file)

# Check and print the lengths of train, validation, and test sets
train_length = len(custom_dataset.get("train", []))
validation_length = len(custom_dataset.get("validation", []))
test_length = len(custom_dataset.get("test", []))

print("Train length:", train_length)
print("Validation length:", validation_length)
print("Test length:", test_length)


# Convert the custom dataset to the required format (adjust as needed)
dataset_dict = DatasetDict({
    "train": Dataset.from_dict({"translation": custom_dataset["train"]}),
    "validation": Dataset.from_dict({"translation": custom_dataset["validation"]}),
    "test": Dataset.from_dict({"translation": custom_dataset["test"]}),
})

# Store the dataset in the variable raw_datasets
raw_datasets = dataset_dict



Train length: 568
Validation length: 18
Test length: 23


In [83]:
from datasets import DatasetDict, Dataset

# Assuming 'data.json' is the name of your custom dataset file
custom_dataset_path = "/content/enmldata.json"  # Update the path accordingly

# Load your custom dataset from the local file
with open(custom_dataset_path, "r", encoding="utf-8") as file:
    custom_dataset = json.load(file)

# Convert the custom dataset to the required format (adjust as needed)
dataset_dict = DatasetDict({
    "train": Dataset.from_dict({"translation": custom_dataset["train"]}),
    "validation": Dataset.from_dict({"translation": custom_dataset["validation"]}),
    "test": Dataset.from_dict({"translation": custom_dataset["test"]}),
})


In [84]:
print(raw_datasets)


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 568
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 18
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 23
    })
})


In [85]:
raw_datasets['train'][1]

{'translation': {'en': 'I want to cancel order',
  'ml': 'എനിക്ക് ഓർഡർ ക്യാൻസൽ ചെയ്യണം'}}

#Preprocessing the data

In [86]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [87]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [4199, 3, 90, 17, 13, 8440, 31, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [88]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[4199, 3, 90, 17, 13, 8440, 31, 0], [342, 17, 635, 8440, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [89]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["പ്രവേശനക്ഷമത എക്സ്പ്ലോറർ"]))

{'input_ids': [[10632, 9678, 72, 8351, 7348, 105, 220, 95, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [90]:
def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]

    print("Inputs:", inputs)
    print("Targets:", targets)

    # Tokenize inputs and targets
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding='max_length')

    # Assign labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]

    print("Model inputs:", model_inputs)

    return model_inputs


In [91]:
preprocess_function(raw_datasets["train"][:2])

Inputs: ['why do you want to cancel?', 'I want to cancel order']
Targets: ['എന്തുകൊണ്ടാണ് നിങ്ങൾ ക്യാൻസൽ ചെയ്യാൻ ആഗ്രഹിക്കുന്നത്?', 'എനിക്ക് ഓർഡർ ക്യാൻസൽ ചെയ്യണം']
Model inputs: {'input_ids': [[940, 87, 16, 543, 8, 16524, 11, 0, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 2

{'input_ids': [[940, 87, 16, 543, 8, 16524, 11, 0, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660], [21, 543, 8, 16524, 1215, 0, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24660, 24

In [92]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/568 [00:00<?, ? examples/s]

Inputs: ['why do you want to cancel?', 'I want to cancel order', 'How to order', 'How do i check my status？', "Why i can't find my order in my account?", 'How do i contact Customer Service about my order？', 'I forgot my password.', 'How to change my account / email adress.', 'Can i register the account with mobile phone number', 'How do i delete my account', "Why i can't pay for my order", "Why i can't find my order in my account?", "Why i can't find my order in my account?", "Why i can't find my order in my account?", "Why i can't find my order in my account?", "I've understood you have a question regarding canceling order and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you.", "I've been informed that you have a question about canceling order . I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need.", 'You

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Inputs: ['Students of the Dattatreya city Municipal corporation secondary school demonstrated their imagination power by creating the fictitious fort "Duttgarh".', 'With encouragement from Principal Sandhya Medpallivaar the teachers and students built the fort out of clay.', 'Rajesh Gavre, the President of the MNPA teachers association, honoured the school by presenting the award.', 'Ramesh Saatpute examined the fort.', 'Students like Nikhil Kavle, Darshan Gedekar, Sahil Meshram participated in building the fort.', "Narender Barai, the President of the District Art Teachers' association, Shekhar Vanskar, a cashier, Ajay Gundamwar, a member of Gajanan Mehar provided guidance to the students.", 'Nagarsevak, Reeta Mule presented messages from well-wishers.', 'Rohtak. Akhil Bhartiya Janwadi Mahila Samiti and DYFI jointly launched a state-wide campaign against corruption in jobs, fraud and hold-ups.', 'Through this state-wide signature campaign, 10 Lakh (1 million)signatures across the stat

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

Inputs: ['The movement is also bolstered by two former U.S. Transportation secretaries, who in a 2011 report urged Congress to move in the pay-per-mile direction.', 'There is going to be a change in how we pay these taxes.', 'The technology is there to do it.', "The push comes as the country's Highway Trust Fund, financed with taxes Americans pay at the gas pump, is broke.", "Americans don't buy as much gas as they used to.", 'Cars get many more miles to the gallon.', "The federal tax itself, 18.4 cents per gallon, hasn't gone up in 20 years.", 'Politicians are loath to raise the tax even one penny when gas prices are high.', '"The gas tax is just not sustainable," said Lee Munnich, a transportation policy expert at the University of Minnesota.', 'His state recently put tracking devices on 500 cars to test out a pay-by-mile system.', '"This works out as the most logical alternative over the long term," he said.', 'Wonks call it a mileage-based user fee.', 'It is no surprise that the id

In [93]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ml.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [94]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [95]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [96]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [97]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)


In [98]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [99]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [117]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [118]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

1/1 [==============================] - 69s 69s/step - loss: 5.7637 - val_loss: 5.4497


In [102]:
model.save_pretrained("tf_model/")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[24660]], 'forced_eos_token_id': 0}


# Model Testing

In [103]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [114]:
input_text  = "I want to order"
tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[24660   366 18492  4803     2     0 24660]], shape=(1, 7), dtype=int32)


In [115]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

എനിക്ക് ഓർഡർ ചെയ്യണം.
